In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!mkdir dataset
!cp /gdrive/MyDrive/Amazon_data/*.csv dataset

In [22]:
import pandas as pd
import numpy as np
from tqdm import tqdm

tqdm.pandas()

In [34]:
train = pd.read_csv("dataset/train_processed.csv")
test = pd.read_csv("dataset/test_processed.csv")

In [35]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2137182 entries, 0 to 2137181
Data columns (total 4 columns):
 #   Column           Dtype  
---  ------           -----  
 0   PRODUCT_TYPE_ID  int64  
 1   PRODUCT_LENGTH   float64
 2   DATA             object 
 3   DATA_L           int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 65.2+ MB


In [36]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 734736 entries, 0 to 734735
Data columns (total 3 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   PRODUCT_TYPE_ID  734736 non-null  int64 
 1   PRODUCT_ID       734736 non-null  int64 
 2   DATA             734733 non-null  object
dtypes: int64(2), object(1)
memory usage: 16.8+ MB


In [37]:
trainpid = set(train["PRODUCT_TYPE_ID"])
testpid = set(test["PRODUCT_TYPE_ID"])

In [38]:
print(len(trainpid), len(testpid), len(trainpid.intersection(testpid)))

12693 10565 10004


In [39]:
len(testpid - trainpid)

561

In [40]:
len(trainpid - testpid)

2689

In [41]:
GLOBAL = train.describe()
LOCAL = {}
for i in tqdm(trainpid) : 
  LOCAL[i] = train[train["PRODUCT_TYPE_ID"] == i].describe()

100%|██████████| 12693/12693 [02:38<00:00, 80.21it/s]


In [42]:
def predict_length(x) :
  id = x["PRODUCT_TYPE_ID"]
  if id in trainpid : 
    return LOCAL[id]["PRODUCT_LENGTH"]["mean"]
  return GLOBAL["PRODUCT_LENGTH"]["mean"]

test["PRODUCT_LENGTH"] = test.progress_apply(predict_length, axis = 1)

100%|██████████| 734736/734736 [00:18<00:00, 39364.43it/s]


In [43]:
test.isna().sum()

PRODUCT_TYPE_ID    0
PRODUCT_ID         0
DATA               3
PRODUCT_LENGTH     0
dtype: int64

In [44]:
test = test[["PRODUCT_ID", "PRODUCT_LENGTH"]]

In [45]:
test.head()

,PRODUCT_ID,PRODUCT_LENGTH
0,604373,824.292091
1,1729783,1167.760455
2,1871949,1345.446886
3,1107571,200.787169
4,624253,678.030933


In [46]:
test.to_csv("submission.csv", index=False)